In [ ]:
#inicializando GEE
try:
    import ee
except ModuleNotFoundError: 
    !pip install earth-engine-api
    import ee
    
try:
    ee.Initialize()
except ee.EEException:
    !earthengine authenticate
    
import geemap
import folium
from ipygee import *
import matplotlib.pyplot as plt
from matplotlib import gridspec
import pandas as pd


# install some packages
#!pip install ipygee

In [ ]:
#polygons loading hardcode
ID411 = r"C:\Users\guilherme.fronza\OneDrive\Amazon_Select\Notebooks\ID_411.shp"
ID373 = r"C:\Users\guilherme.fronza\OneDrive\Amazon_Select\Notebooks\ID_373.shp"
ID483 = r"C:\Users\guilherme.fronza\OneDrive\Amazon_Select\Notebooks\ID_483.shp"

#create ee.Feature with GEEMAP
pol = geemap.shp_to_ee(ID483)

#NDVI to Landsat-8
def ndvi_func_LC8(image):
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

#NDVI to Sentinel-2
def ndvi_func_S2(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

#Modis scale factor function
def scale_factor_MODIS(image):
    # scale factor for the MODIS MOD13Q1 product
    return image.multiply(0.0001).copyProperties(image, ['system:time_start'])

##Mascará de nuvens para a banda pixel_qa SR landsat 4,5 e 7
def cloudMaskL457(image):
    qa = image.select('pixel_qa') ##substitiu a band FMASK
    cloud1 = qa.bitwiseAnd(1<<5).eq(0)
    cloud2 = qa.bitwiseAnd(1<<7).eq(0)
    cloud3 = qa.bitwiseAnd(1<<3).eq(0)
    mask2 = image.mask().reduce(ee.Reducer.min());
    return image.updateMask(cloud1).updateMask(cloud2).updateMask(cloud3).updateMask(mask2).divide(10000).copyProperties(image, ["system:time_start"])

# Define a cloud masking function.
def maskL8sr(image):
    cloudShadowBitMask = 1<<3
    cloudBitMask = 1<<5
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudBitMask).eq(0))
    return image.updateMask(mask)


#Visualization with Folium Map GEE integrated
def add_ee_layer(self, ee_object, vis_params, name):
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            fill = False,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    except:
        print("Could not display {}".format(name))

#Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

polygon_params = {
  'palette': ['red']}
location = pol.geometry().centroid(10).coordinates().getInfo()[::-1]
Map = folium.Map(location=location, zoom_start=18)
basemaps['Google Satellite Hybrid'].add_to(Map)
Map.add_ee_layer(pol, polygon_params, 'polygon')
loc = '-'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)  
Map.get_root().html.add_child(folium.Element(title_html))
Map
#filename_localizacao = 'Localizacao.html'
#Map.save(filename_localizacao)

# # Image collection build, spatial and temporal filter
# l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
#   .filterBounds(pol) \
#   .filterDate('2015-01-01', '2022-12-31').map(maskL8sr)

# count = l8sr.size().getInfo();
# print('Images count: ', count)


In [ ]:
# def index_dataframe(imagecoll, polygon, scale):
#     #time series
#     polygon_ndvi = chart.Image.series(**{'imageCollection': imagecoll,
#                                        'region': polygon,
#                                        'reducer': ee.Reducer.mean(),
#                                        'scale': scale,
#                                        'xProperty': 'system:time_start'})
#     df = polygon_ndvi.dataframe
#     return df

def NDVI_mean_reducer(periods_NDVI, collections_NDVI, polygon, scale):
    columns = ['Período', 'NDVI']
    data_mean = []
    for collection_ndvi, period in zip(collections_NDVI, periods_NDVI):
        print(period)
        mean = collection_ndvi.select('NDVI').reduceRegion(**{
          'reducer': ee.Reducer.mean(),
          'geometry': polygon,
          'scale': scale,
          'crs': 'EPSG:4326',
          'maxPixels': 1e9
                 })
        data_mean.append({
         columns[0]: period,
         columns[1]: mean.getInfo()['NDVI']
     })
    df_polygon_NDVI_mean = pd.DataFrame(data_mean)
    #df_polygon_NDVI_mean = df_polygon_NDVI_mean.T
    #my_series = df_polygon_NDVI_mean.values.tolist()
    #means = my_series[1]
    return df_polygon_NDVI_mean

def forest_restauration_periods_NDVI_median(interest_period_dict, geometry):
    periods = list()
    collections = list()
    for period in interest_period_dict:
        start_date = interest_period_dict[period]['start_date']
        end_date = interest_period_dict[period]['end_date']
        
        l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
          .filterBounds(geometry) \
          .filterDate(start_date, end_date).map(maskL8sr) \
          .sort('system:time_start')
        a = l8sr.count().getInfo()
        l8sr_ndvi_med = l8sr.map(ndvi_func_LC8).select('NDVI').median()
        periods.append(period)
        collections.append(l8sr_ndvi_med)
        
    return periods, collections, a

def forest_restauration_periods_NDVI_max(interest_period_dict, geometry):
    periods = list()
    collections = list()
    for period in interest_period_dict:
        start_date = interest_period_dict[period]['start_date']
        end_date = interest_period_dict[period]['end_date']
        
        l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
          .filterBounds(geometry) \
          .filterDate(start_date, end_date).map(maskL8sr) \
          .sort('system:time_start')
        l8sr_ndvi_med = l8sr.map(ndvi_func_LC8).select('NDVI').max()
        periods.append(period)
        collections.append(l8sr_ndvi_med)
    return periods, collections

def plot_NDVI_series(df, identif):
    fig, ax = plt.subplots(figsize=(20, 10))
    #fig.suptitle('Stack de plots de séries temporais', size=20)
    #ts1
    ax.plot(df.Período, df['NDVI'],
            color='green')
    ax.set_title(identif, size = 15)
    plt.ylim(0.3, 1.0)
    plt.xticks(rotation=45, size=13)
    return fig 

interest_period_dict_semestral_7years = {
                            'landsat2015_1': {'start_date':"2015-01-01",'end_date':"2015-06-30"},
                            'landsat2015_2': {'start_date':"2015-07-01",'end_date':"2015-12-31"},
                            'landsat2016_1': {'start_date':"2016-01-01",'end_date':"2016-06-30"},
                            'landsat2016_2': {'start_date':"2016-07-01",'end_date':"2016-12-31"},
                            'landsat2017_1': {'start_date':"2017-01-01",'end_date':"2017-06-30"},
                            'landsat2017_2': {'start_date':"2017-07-01",'end_date':"2017-12-31"},
                            'landsat2018_1': {'start_date':"2018-01-01",'end_date':"2018-06-30"},
                            'landsat2018_2': {'start_date':"2018-07-01",'end_date':"2018-12-31"},
                            'landsat2019_1': {'start_date':"2019-01-01",'end_date':"2019-06-30"},
                            'landsat2019_2': {'start_date':"2019-07-01",'end_date':"2019-12-31"},
                            'landsat2020_1': {'start_date':"2020-01-01",'end_date':"2020-06-30"},
                            'landsat2020_2': {'start_date':"2020-07-01",'end_date':"2020-12-31"},
                            'landsat2021_1': {'start_date':"2021-01-01",'end_date':"2021-06-30"},
                            'landsat2021_2': {'start_date':"2021-07-01",'end_date':"2021-12-31"}
                }

#Collection .median() aggregation
periods_LC8_median, collections_LC8_median = forest_restauration_periods_NDVI_median(interest_period_dict_semestral_7years, geometry = pol)
df_median = NDVI_mean_reducer(periods_NDVI=periods_LC8_median, collections_NDVI=collections_LC8_median, polygon=pol, scale=30)
fig_median = plot_NDVI_series(df=df_median, identif='ID Plantio 483 Cacau Floresta - Median')


In [ ]:
#Collection .max() aggregation
periods_LC8_max, collections_LC8_max = forest_restauration_periods_NDVI_max(interest_period_dict_semestral_7years, geometry = pol)
df_max = NDVI_mean_reducer(periods_NDVI=periods_LC8_max, collections_NDVI=collections_LC8_max, polygon=pol, scale=30)
fig_max = plot_NDVI_series(df=df_max, identif='ID Plantio 483 Cacau Floresta - Max')

In [ ]:
def TS_NDVI_plot(imagecoll, polygon, filter_param, scale):
    #time series
    polygon_ndvi = chart.Image.series(**{'imageCollection': imagecoll.select('NDVI'),
                                       'region': polygon,
                                       'reducer': ee.Reducer.mean(),
                                       'scale': scale,
                                       'xProperty': 'system:time_start'})
    df = polygon_ndvi.dataframe
    df = df.rolling(filter_param, min_periods=1).mean()
    return df


# Image collection build, spatial and temporal filter
l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
  .filterBounds(pol) \
  .filterDate('2015-01-01', '2022-12-31').map(maskL8sr)

#NDVI 
images_ndvi_LC8 = l8sr.map(ndvi_func_LC8)

#TS Constructors
LC8_NDVI_ts = TS_NDVI_plot(images_ndvi_LC8, pol, filter_param=2, scale=30)

In [ ]:
def plot_NDVI_series_full(df, identif):
    fig, ax = plt.subplots(figsize=(20, 10))
    #fig.suptitle('Stack de plots de séries temporais', size=20)
    #ts1
    ax.plot(df.index, df['NDVI'],
            color='green')
    ax.set_title(identif, size = 15)
    plt.ylim(0.2, 1.0)
    plt.xticks(rotation=45, size=13)
    return fig 

figura_TS_LC8 = plot_NDVI_series_full(df=LC8_NDVI_ts, identif='ID Plantio 373 Cacau Floresta - LANDSAT-8 OLI NDVI TIME SERIES')

In [ ]:
#test ndvi image aggregated visualization
Map_ndvi = folium.Map(location=location, zoom_start=15)
ndviParams = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}
Map_ndvi.add_ee_layer(collections_LC8[0], ndviParams, 'NDVI image')
Map_ndvi.add_ee_layer(pol, polygon_params, 'polygon')
Map_ndvi

In [ ]:
def get_s2_sr_cld_col(polygon, start_date, end_date):
    CLOUD_FILTER = 90
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(polygon)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(polygon)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

def add_cld_shdw_mask(img):
    BUFFER = 20
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focal_min(2).focal_max(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

def add_cloud_bands(img):
    CLD_PRB_THRESH = 40
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_shadow_bands(img):
   
    NIR_DRK_THRESH = 0.15
    CLD_PRJ_DIST = 2
   
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

#plot all timeseries
def TS_NDVI_plot(imagecoll, polygon, filter_param, scale):
    #time series
    polygon_ndvi = chart.Image.series(**{'imageCollection': imagecoll.select('NDVI'),
                                       'region': polygon,
                                       'reducer': ee.Reducer.mean(),
                                       'scale': scale,
                                       'xProperty': 'system:time_start'})
    df = polygon_ndvi.dataframe
    df = df.rolling(filter_param, min_periods=1).mean()
    return df

def plot_LC8_S2_MODIS(ts1,ts2,ts3, ts1_name, ts2_name, ts3_name):
    #Plot three TS - LANDSAT8/S2/MODIS
   
    fig, axs = plt.subplots(3, figsize=(10, 15))
    fig.suptitle('Stack de plots de séries temporais', size=20)
    #ts1
    axs[0].plot(ts1.index, ts1['NDVI'],
            color='green')
    axs[0].set_title(ts1_name, size = 15)
    axs[0].set_ylim(0.0, 1.0)
    axs[0].spines['right'].set_visible(False)
    axs[0].spines['top'].set_visible(False)

    #ts2
    axs[1].plot(ts2.index, ts2['NDVI'],
            color='green')
    axs[1].set_title(ts2_name, size = 15)
    axs[1].set_ylim(0.0, 1.0)
    axs[1].spines['right'].set_visible(False)
    axs[1].spines['top'].set_visible(False)
   
    #ts3
    axs[2].plot(ts3.index, ts3['NDVI'],
            color='green')
    axs[2].set_title(ts3_name, size = 15)
    axs[2].set_ylim(0.0, 1.0)
    axs[2].spines['right'].set_visible(False)
    axs[2].spines['top'].set_visible(False)
   
    fig.tight_layout(rect=[0, 0.03, 1, 0.98])
    gs = gridspec.GridSpec(29, 21, wspace=0.1, hspace=0.1)
    gs.tight_layout(fig, pad=0)
    return fig

#collections 2015 - 2021
MOD13Q1 = (ee.ImageCollection('MODIS/006/MOD13Q1')
                .filterBounds(pol)
                .filterDate(ee.DateRange('2010-01-01','2022-12-31'))
                .sort('system:time_start'))

# Image collection build, spatial and temporal filter
l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
  .filterBounds(pol) \
  .filterDate('2015-01-01', '2022-12-31').map(maskL8sr)

#Sentinel-2 TS filter
s2_sr_cld_col = get_s2_sr_cld_col(polygon=pol, start_date='2015-01-01', end_date='2022-12-31')
s2_sr = (s2_sr_cld_col.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask).map(ndvi_func_S2))
                             #.median())

#map indices - apply ndvi_func
images_ndvi_LC8 = l8sr.map(ndvi_func_LC8)
#images_ndvi_S2 = s2_sr_median.map(ndvi_func_S2)

#NDVI selection and scale factor to MODIS
ndvi_modis = MOD13Q1.select('NDVI')

# mapping function to multiply by the scale factor
images_ndvi_modis_scaled = ndvi_modis.map(scale_factor_MODIS)

#TS Constructors
LC8_NDVI_ts = TS_NDVI_plot(images_ndvi_LC8, pol, filter_param =5, scale=30)
S2_NDVI_ts = TS_NDVI_plot(s2_sr, pol, filter_param =3, scale=10)
MOD13Q1_NDVI_ts = TS_NDVI_plot(images_ndvi_modis_scaled, pol, filter_param = 4, scale=250)


Figura_TS = plot_LC8_S2_MODIS(LC8_NDVI_ts, S2_NDVI_ts, MOD13Q1_NDVI_ts, ts1_name = 'LANDSAT-8 OLI NDVI TIME SERIES',
                  ts2_name = 'SENTINEL-2 MSI NDVI TIME SERIES', ts3_name = 'MODIS MOD13Q1 NDVI TIME SERIES')